In [26]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams["figure.figsize"] = (7,4)


In [27]:
trades = pd.read_csv("historical_data.csv")
sentiment = pd.read_csv("fear_greed_index.csv")


In [28]:
print("Trades columns:", list(trades.columns))
print("Sentiment columns:", list(sentiment.columns))

trades.head()
sentiment.head()


Trades columns: ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']
Sentiment columns: ['timestamp', 'value', 'classification', 'date']


,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


In [29]:
trades.columns = (
    trades.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

sentiment.columns = (
    sentiment.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

print("Cleaned trades columns:", list(trades.columns))
print("Cleaned sentiment columns:", list(sentiment.columns))


Cleaned trades columns: ['account', 'coin', 'execution_price', 'size_tokens', 'size_usd', 'side', 'timestamp_ist', 'start_position', 'direction', 'closed_pnl', 'transaction_hash', 'order_id', 'crossed', 'fee', 'trade_id', 'timestamp']
Cleaned sentiment columns: ['timestamp', 'value', 'classification', 'date']


In [30]:

sentiment = sentiment.loc[:, ~sentiment.columns.duplicated()]

print("Sentiment columns after removing duplicates:", list(sentiment.columns))


Sentiment columns after removing duplicates: ['timestamp', 'value', 'classification', 'date']


In [31]:
# Case 1: classification already exists
if "classification" in sentiment.columns:
    pass

# Case 2: numeric sentiment index exists
elif "value" in sentiment.columns:
    sentiment["classification"] = sentiment["value"].apply(
        lambda x: "Fear" if x < 50 else "Greed"
    )

else:
    raise ValueError("No usable sentiment column found")


In [32]:
# Verify (THIS MUST WORK)
sentiment["classification"].value_counts()


classification
Fear             781
Greed            633
Extreme Fear     508
Neutral          396
Extreme Greed    326
Name: count, dtype: int64

In [33]:
sentiment["date"] = pd.to_datetime(sentiment["date"]).dt.date


In [34]:
# Detect timestamp column automatically
time_col = None
for col in trades.columns:
    if "time" in col or "date" in col:
        time_col = col
        break

print("Detected time column in trades:", time_col)

if time_col is None:
    raise ValueError("No time/date column found in trades dataset")


Detected time column in trades: timestamp_ist


In [36]:
# ✅ FIX datetime parsing (error-free for your dataset)
trades[time_col] = pd.to_datetime(
    trades[time_col],
    dayfirst=True,   # REQUIRED for DD-MM-YYYY
    errors="coerce"  # Prevents crashes
)

# Create date column for merging
trades["date"] = trades[time_col].dt.date


In [37]:
trades[[time_col, "date"]].head()


,timestamp_ist,date
0,2024-12-02 22:50:00,2024-12-02
1,2024-12-02 22:50:00,2024-12-02
2,2024-12-02 22:50:00,2024-12-02
3,2024-12-02 22:50:00,2024-12-02
4,2024-12-02 22:50:00,2024-12-02


In [38]:
sentiment["date"] = pd.to_datetime(sentiment["date"]).dt.date


In [39]:
merged = trades.merge(
    sentiment[["date", "classification"]],
    on="date",
    how="left"
)

merged.head()


,account,coin,execution_price,size_tokens,size_usd,side,timestamp_ist,start_position,direction,closed_pnl,transaction_hash,order_id,crossed,fee,trade_id,timestamp,date,classification
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,2024-12-02 22:50:00,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12,2024-12-02,Extreme Greed
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,2024-12-02 22:50:00,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12,2024-12-02,Extreme Greed
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,2024-12-02 22:50:00,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12,2024-12-02,Extreme Greed
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,2024-12-02 22:50:00,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12,2024-12-02,Extreme Greed
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,2024-12-02 22:50:00,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12,2024-12-02,Extreme Greed


In [40]:
# Check how many rows have sentiment info
merged["classification"].value_counts(dropna=False)


classification
Fear             61837
Greed            50303
Extreme Greed    39992
Neutral          37686
Extreme Fear     21400
NaN                  6
Name: count, dtype: int64

In [41]:
merged.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211224 entries, 0 to 211223
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   account           211224 non-null  object        
 1   coin              211224 non-null  object        
 2   execution_price   211224 non-null  float64       
 3   size_tokens       211224 non-null  float64       
 4   size_usd          211224 non-null  float64       
 5   side              211224 non-null  object        
 6   timestamp_ist     211224 non-null  datetime64[ns]
 7   start_position    211224 non-null  float64       
 8   direction         211224 non-null  object        
 9   closed_pnl        211224 non-null  float64       
 10  transaction_hash  211224 non-null  object        
 11  order_id          211224 non-null  int64         
 12  crossed           211224 non-null  bool          
 13  fee               211224 non-null  float64       
 14  trad

## Analysis 5: Buy vs Sell Performance

This analysis compares buy and sell performance under different market
sentiment conditions.

**Observation:**  
Performance differs between buy and sell trades depending on sentiment.

**Interpretation:**  
Sentiment-aware trade direction strategies may improve results.
